### 첫 번째 적 우주선 배치

적 우주선의 동작은 Enemy 클래스에서 당담합니다. 구조는 ship 클래스와 유사합니다. 적 우주선 이미지를 다운 받아 images 폴더에 저장합니다. 

In [ ]:
import pygame
from pygame.sprite import Sprite

class Enemy(Sprite):

    def __init__(self, uw_game):
        """ 외계인 우주선을 초기화하고 시작 위치를 정합니다. """
        super().__init__()
        self.screen = uw_game.screen

        # 외계인 이미지를 불러오고 객체의 rect 속성을 설정합니다.
        self.image = pygame.image.load('images/enemy.png')
        self.rect = self.image.get_rect()

        # 외계인을 화면 좌측 상단에 배치합니다. 
        self.rect.x = self.rect.width
        self.rect.y = self.rect.height

        self.x = float(self.rect.x)

### Enemy 인스턴스 생성

적 우주선을 화면에 표시하려면 Enemy 인스턴스를 생성해야 합니다. 인스턴스를 만드는 작업은 초기화 작업에 해당하므로 UniverseWar 클래스의 생성자 마지막 줄에 추가합니다. 최종적으로는 함대 전체를 생성해야 하므로 새 보조 메서드 \_create_fleet() 도 작성하겠습니다.

먼저 Enemy 클래스를 임포트합니다. 

In [1]:

# universe_war.py

import sys
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정

        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        self.enemies.add(enemy)





    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self.bullets.update()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._update_screen()
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


ModuleNotFoundError: No module named 'settings'

### 적 함대 만들기

함대를 그리려면 적 우주선이 화면에 몇 줄 있어야 할지, 한 줄에는 우주선이 몇 대가 있어야 할 지 계산해야 합니다. 첫 번째 적 우주선 사이의 가로 공간을 결정하고 한 줄을 만든 다음 줄 사이의 세로 공간을 결정해서 함대 전체를 만듭니다. 

### 적 우주선이 한 줄에 얼마나 들어갈지 계산

화면 너비는 settings.screen_width 에 저장되어 있지만 화면의 양쪽에 마진(빈 공간)이 어느 정도 있어야 합니다. 가로 마진은 적 우주선 하나이 너비로 정합니다. 좌우에 마진이 하나 있으니, 가용 공간은 화면 너비에서 외계인 그림 너비의 두 배를 뺀 만큼으로 정합니다. 


~~~ python
available_space_x = settings.screen_width - (2 * enemy_width)  # 화면 양쪽의 마진을 뺀 공간
~~~

적 우주선 사이에도 공간이 필요합니다. 이 공간은 적 우주선 하나의 너비로 정합니다. 따라서 적 우주선 하나를 표시하는데 필요한 공간은 적 우주선 너비의 두배 입니다. 한 줄에 들어갈 적 우주선 숫자는 가용공간을 적 우주선 너비의 두 배로 나누면 됩니다. 나눌 때는 몫만 구하기 위해서 //연산자를 사용합니다. 

~~~python
number_enemy_x = available_space_x // (2 * enemy_width)
~~~

### 적 우주선 한 줄 만들기 

_create_fleet() 메서드를 다음과 같이 수정하여 적 우주선을 한 줄 생성합니다. 

In [ ]:

# universe_war.py

import sys
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정

        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수


        for enemy_number in range(number_enemy_x):
            self._create_enemy(enemy_number)
        

    def _create_enemy(self, enemy_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x
        self.enemies.add(enemy)





    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self.bullets.update()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._update_screen()
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game

### 줄 추가하기 

화면에 적 우주전 몇 줄을 넣을지 결정하고 그 숫자만큼 적 우주선을 한 줄 만드는 코드를 반복하면 함대가 완성됩니다. 줄 숫자를 결정하려면 화면 높이에서 적 우주선 하나의 높이, 플레이어 우주선 높이, 적 우주선 높이의 두배를 빼서 사용할 수 있는 세로 공간을 계산해야 합니다.

~~~python
available_space_y = settings.screen_height - (3 * enemy_height) - ship_height 
~~~

각 줄 사이에도 공간이 필요합니다. 이 공간은 적 우주선 하나의 높이로 합니다. 몇 줄을 만들지 계산하려면 가용 공간을 적 우주선 높이의 두 배로 나누면 됩니다.

~~~python
number_rows = avilable_space_y // (2 * enemy_height)
~~~


In [ ]:

# universe_war.py

import sys
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정

        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self.bullets.update()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._update_screen()
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game

In [ ]:

# universe_war.py

import sys
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정

        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self._update_bullets()
            self._update_enemies()
            self._update_screen()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._update_screen()

    def _update_enemies(self):
        self.enemies.update()
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game


### 세팅 클래스에 함대 방향 속성 추가

함대가 화면 오른쪽 경계에 닿으면 아래로 움직인 다음 왼쪽으로 방향을 바꾸는 세팅을 저장할 것입니다. 다음과 같이 Settings 클래스에 속성을 추가합니다. 

In [ ]:
class Settings:
    """게임의 세팅을 저장하는 클래스"""

    def __init__(self):
        # 화면 세팅
        self.screen_width = 1200
        self.screen_height = 800
        self.bg_color = (230, 230, 230)

        # 우주선 세팅
        self.ship_speed = 1.5

        # 탄환 세팅
        self.bullet_speed = 1.0
        self.bullet_width = 3
        self.bullet_height = 15
        self.bullet_color = (60, 60, 60)
        self.bullets_allowed = 3     #

        # 적 함대 세팅
        self.enemy_speed = 1.0
        self.fleet_drop_speed = 10  # 함대가 경계에 닿았을 때 내려올 위치좌표랑 
        self.fleet_direction = 1    # fleet_direction이 1이면 오른쪽, -1이면 왼쪽으로 이동 

### 적 우주선이 화면 경계에 닿았는지 체크 

enemy.py 모듈의 update() 메서드를 수정해서 적 우주선이 적절한 방향으로 움직이게 합니다. 

In [ ]:
import pygame
from pygame.sprite import Sprite

class Enemy(Sprite):

    def __init__(self, uw_game):
        """ 외계인 우주선을 초기화하고 시작 위치를 정합니다. """
        super().__init__()
        self.screen = uw_game.screen
        self.settings = uw_game.settings

        # 외계인 이미지를 불러오고 객체의 rect 속성을 설정합니다.
        self.image = pygame.image.load('images/enemy.png')
        self.rect = self.image.get_rect()

        # 외계인을 화면 좌측 상단에 배치합니다. 
        self.rect.x = self.rect.width
        self.rect.y = self.rect.height

        self.x = float(self.rect.x)

    def check_edges(self):
        """적 우주선이 화면 경계에 닿으면 True를 반환합니다."""
        screen_rect = self.screen.get_rect()

        if self.rect.right >= screen_rect.right or self.rect.left <= 0:
            return True

    def update(self):
        """적 우주선을 오른쪽이나 왼쪽으로 움직입니다."""
        self.x += (self.settings.enemy_speed * self.settings.fleet_direction)
        self.rect.x = self.x
        

### 함대를 아래로 내리고 방향 바꾸기

적 우주선이 화면 경계에 닿으면 함대 전체가 아래로 내려온 다음 이동 방향을 바꿔야 합니다.
UniverseWar 클래스에 왼쪽이나 오른쪽 경계에 닿은 적이 있는지 체크하는 코드를 추가합니다. 이 작업은 _check_fleet_edges()와 _change_fleet_direction() 메서드에서 담당합니다. 또한 _update_enemies() 도 수정합니다. 

In [ ]:

# universe_war.py

import sys
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정

        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self._update_bullets()
            self._update_enemies()
            self._update_screen()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._update_screen()

    def _update_enemies(self):
        self._check_fleet_edges()
        self.enemies.update()
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _check_fleet_edges(self):
        """ 적 우주선이 화면 경계에 닿았다면 그에 맞게 반응합니다."""
        for enemy in self.enemies.sprites():
            if enemy.check_edges():
                self._change_fleet_direction()
                break

    def _change_fleet_direction(self):
        for enemy in self.enemies.sprites():
            enemy.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction * = -1

    

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game


### 적 우주선 격추하기 

게임 프로그래밍에서 충돌은 게임 요소가 겹칠 때 일어납니다. 탄환이 적 우주선을 격추하게 만들려면 sprite.groupcollide() 메서드를 호출해서 두 그룹의 요소 사이에 충돌이 있는지 확인하면 됩니다. 

### 탄환 출돌 감지

탄환이 적 우주선에 닿으면 적 우주선을 사라지게 해야 합니다. 이를 위해서는 탄환의 위치를 업데이트 하는 즉시 충돌한 객체가 있는지 확인해야 합니다. 

sprite.groupcollide() 메서드는 A 그룹의 각 요소 사각형(rect)을 B 그룹의 각 요소 사각형과 비교하는 방식으로 충돌을 감지합니다. 각 탄환의 rect와 각 적 우주선의 rect를 비교하고 충돌 한 탄환과 적 우주선을 담은 딕셔너리를 반환합니다. 이 딕셔너리의 키는 탄환이고 대응하는 값은 적 우주선입니다. 

다음 코드를 _update_bullets() 메서드의 마지막에 추가해서 충돌을 감지합니다. 

### 함대 다시 생성하기

함대를 섬멸한 후 새로운 함대가 나타나게 하려면 enemies 그룹이 비어 있는지 체크해야 합니다.
비어 있으면 _create_fleet()을 호출해서 새 함대를 생성합니다. 적 우주선을 파괴하는 루틴은 _update_bullets() 메서드이므로 체크 역시 이 메서드의 마지막에서 해야 합니다. 

In [ ]:

# universe_war.py

import sys
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정

        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self._update_bullets()
            self._update_enemies()
            self._update_screen()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            collisions = pygame.sprite.groupcollide(self.bullets, self.enemies, True, True)

            

    def _update_enemies(self):
        self._check_fleet_edges()
        self.enemies.update()
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _check_fleet_edges(self):
        """ 적 우주선이 화면 경계에 닿았다면 그에 맞게 반응합니다."""
        for enemy in self.enemies.sprites():
            if enemy.check_edges():
                self._change_fleet_direction()
                break

    def _change_fleet_direction(self):
        for enemy in self.enemies.sprites():
            enemy.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction * = -1

    

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game


### _update_bullets() 리팩토링

함수가 너무 많은 작업을 하고 있으므로 리팩토링을 실시해서 단순하게 바꿉니다. 탄환과 적 우주선의 충돌을 담당하는 코드를 별도의 메서드로 추출합니다. 

In [ ]:

# universe_war.py

import sys
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정

        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self._update_bullets()
            self._update_enemies()
            self._update_screen()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._check_bullet_enemy_collisions()

           
    def _check_bullet_enemy_collisions(self):
         
        collisions = pygame.sprite.groupcollide(self.bullets, self.enemies, True, True)

            
        
            if not self.enemies:
                self.bullets.empty()
                self._create_fleet()

        

            

    def _update_enemies(self):
        self._check_fleet_edges()
        self.enemies.update()
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _check_fleet_edges(self):
        """ 적 우주선이 화면 경계에 닿았다면 그에 맞게 반응합니다."""
        for enemy in self.enemies.sprites():
            if enemy.check_edges():
                self._change_fleet_direction()
                break

    def _change_fleet_direction(self):
        for enemy in self.enemies.sprites():
            enemy.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction * = -1

    

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game


### 게임 끝내기

플레이어가 함대를 제 시간 내에 섬멸하지 못하면 적 우주선이 플레이어 우주선에 충돌해 파괴하게 만들겠습니다. 또한 플레이어가 사용할 수 있는 우주선 숫자도 제한하고 적 우주선이 화면 아래쪽에 닿아도 플레이어 우주선을 파괴합니다. 플레이어가 우주선을 전부 잃으면 게임이 끝납니다. 

### 적 우주선과 플레이어 우주선 충돌 감지
적 우주선과 플레이어 우주선의 충돌은 UniverseWar 클래스에서 각각의 적 우주선의 위치를 업데이트한 바로 다음에 체크하면 됩니다.

In [ ]:

# universe_war.py

import sys
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정

        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self._update_bullets()
            self._update_enemies()
            self._update_screen()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._check_bullet_enemy_collisions()

           
    def _check_bullet_enemy_collisions(self):
         
        collisions = pygame.sprite.groupcollide(self.bullets, self.enemies, True, True)

                        
        
            if not self.enemies:
                self.bullets.empty()
                self._create_fleet()

        

            

    def _update_enemies(self):
        self._check_fleet_edges()
        self.enemies.update()

        # 적 우주선이 플레이어 우주선과 충돌했는지 확인합니다.
        if pygame.sprite.spritecollideany(self.ship, self.enemies):
            print("ship hit!!!")
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _check_fleet_edges(self):
        """ 적 우주선이 화면 경계에 닿았다면 그에 맞게 반응합니다."""
        for enemy in self.enemies.sprites():
            if enemy.check_edges():
                self._change_fleet_direction()
                break

    def _change_fleet_direction(self):
        for enemy in self.enemies.sprites():
            enemy.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction * = -1

    

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game


### 적 우주선과 플레이어 우주선 충돌 처리

적 우주선과 플레이어 우주선이 충돌했을 때 ship 인스턴스를 파괴하고 새로 만들기보다는 게임 기록을 추적해서 플레이어 우주선이 얼마나 자주 파괴됐는지 알아보는게 좋습니다.

다음과 같이 game_stats.py 모듈을 생성하고 게임 기록을 저장할 GameStats 클래스를 정의합니다.

In [ ]:
# game_stats.py

class GameStats:
    """ 게임 기록 저장 """
    def __init__(self, uw_game):
        """ 기록 초기화 """
        self.settings = uw_game.settings
        self.reset_stats()

    def reset_stats(self):
        """ 게임을 진행하는 동안 바뀔 수 있는 기록을 초기화합니다. """
        self.ships_left = self.settings.ship_limit

GameStats 인스턴스를 생성하기 위해서 임포트 합니다. (Universe_war.py)

In [ ]:

# universe_war.py

import sys
from time import sleep 
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy
from game_stats import GameStats

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정

        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self._update_bullets()
            self._update_enemies()
            self._update_screen()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._check_bullet_enemy_collisions()

           
    def _check_bullet_enemy_collisions(self):
         
        collisions = pygame.sprite.groupcollide(self.bullets, self.enemies, True, True)

                        
        
            if not self.enemies:
                self.bullets.empty()
                self._create_fleet()

        

            

    def _update_enemies(self):
        self._check_fleet_edges()
        self.enemies.update()

        # 적 우주선이 플레이어 우주선과 충돌했는지 확인합니다.
        if pygame.sprite.spritecollideany(self.ship, self.enemies):
            print("ship hit!!!")
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _check_fleet_edges(self):
        """ 적 우주선이 화면 경계에 닿았다면 그에 맞게 반응합니다."""
        for enemy in self.enemies.sprites():
            if enemy.check_edges():
                self._change_fleet_direction()
                break

    def _change_fleet_direction(self):
        for enemy in self.enemies.sprites():
            enemy.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction * = -1

    

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game()

UniverseWar 클래스의 __init__() 에서 GameStats 인스턴스를 생성합니다. 

In [ ]:

# universe_war.py

import sys
from time import sleep 
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy
from game_stats import GameStats

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정
        
        self.stats = GameState(self)
        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self._update_bullets()
            self._update_enemies()
            self._update_screen()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._check_bullet_enemy_collisions()

           
    def _check_bullet_enemy_collisions(self):
         
        collisions = pygame.sprite.groupcollide(self.bullets, self.enemies, True, True)

                        
        
            if not self.enemies:
                self.bullets.empty()
                self._create_fleet()

        

            

    def _update_enemies(self):
        self._check_fleet_edges()
        self.enemies.update()

        # 적 우주선이 플레이어 우주선과 충돌했는지 확인합니다.
        if pygame.sprite.spritecollideany(self.ship, self.enemies):
            print("ship hit!!!")
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _check_fleet_edges(self):
        """ 적 우주선이 화면 경계에 닿았다면 그에 맞게 반응합니다."""
        for enemy in self.enemies.sprites():
            if enemy.check_edges():
                self._change_fleet_direction()
                break

    def _change_fleet_direction(self):
        for enemy in self.enemies.sprites():
            enemy.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction * = -1

    

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game()

이 인스턴스는 게임 창이 만들어진 뒤, 우주선 같이 게임 요소를 정의하기 위해 전에 생성해야 합니다.

적 우주선이 플레이어 우주선에 충돌하면 남아 있는 우주선 숫자를 하나 줄이고, 적 우주선과 탄환을 모두 제거하고, 함대를 새로 만들고, 플레이어 우주선을 화면 하단 중앙에 배치합니다. 또 게임을 일시정지해서 플레이어가 우주선을 잃은 것을 알아차리고 새 함대가 등장하기 전의 준비 시간을 줍니다.

이 코드들을 새 메서드 _ship_hit()으로 작성합니다. 이 메서드를 _update_enemies()에서 적 우주선이 충돌했을 때 호출합니다. 

In [ ]:

# universe_war.py

import sys
from time import sleep 
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy
from game_stats import GameStats

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정
        
        self.stats = GameStates(self)
        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            self.ship.update()
            self._update_bullets()
            self._update_enemies()
            self._update_screen()
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._check_bullet_enemy_collisions()

           
    def _check_bullet_enemy_collisions(self):
         
        collisions = pygame.sprite.groupcollide(self.bullets, self.enemies, True, True)

                        
        
            if not self.enemies:
                self.bullets.empty()
                self._create_fleet()

    
    # 적 우주선이 바닥에 닿았는지 확인하고 처리합니다.
    def _check_enemies_bottom(self):
        screen_rect = self.screen.get_rect()
        for enemy in self.enemies.sprites():
            if enemy.rect.bottom >= screen_rect.bottom:
                self.ship_hit()
                break

       

    def _update_enemies(self):
        self._check_fleet_edges()
        self.enemies.update()

        # 적 우주선이 플레이어 우주선과 충돌했는지 확인합니다.
        if pygame.sprite.spritecollideany(self.ship, self.enemies):
            self._ship_hit()

        self._check_enemies_bottom()

    def _ship_hit(self):
        """플레이어 우주선이 적 우주선과 충돌했을 떄 반응합니다."""
        # 플레이어 우주선 갯수를 줄입니다. 
        self.stats.ships_left -= 1

        # 남아 있는 적 우주선과 탄환을 제거합니다.
        self.enemies.empty()
        self.bullets.empty()

        # 새 함대를 만들고 플레이어 우주선을 배치합니다.
        self._create_fleet()
        self.ship.center_ship()

        # 일시 정지
        sleep(0.5)
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _check_fleet_edges(self):
        """ 적 우주선이 화면 경계에 닿았다면 그에 맞게 반응합니다."""
        for enemy in self.enemies.sprites():
            if enemy.check_edges():
                self._change_fleet_direction()
                break

    def _change_fleet_direction(self):
        for enemy in self.enemies.sprites():
            enemy.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction * = -1

    

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game()

### 적 우주선이 화면 아래쪽에 도착했을 때

적 우주선이 화면 아래쪽에 도착했을 때도 플레이어 우주선에 부딪혔을 때와 마찬가지로 반응해야 합니다. 화면 아래쪽에 닿은 적 우주선이 있는지 알아보기 위해서 새 매서드를 추가하겠습니다. 

## 게임오버 

게임은 아직 끝나지 않습니다. ship_left 값은 계속 마이너스 방향으로 증가합니다. GameStats에 game_active 플래그를 추가하여 플레이어 우주선을 모두 잃었을 때 게임을 끝내도록 처리합니다. 이 플래그 GameStats 클래스의 __init__() 메서드 마지막에 추가합니다.

In [ ]:
# game_stats.py

class GameStats:
    """ 게임 기록 저장 """
    def __init__(self, uw_game):
        """ 기록 초기화 """
        self.settings = uw_game.settings
        self.reset_stats()

        self.game_active = True

    def reset_stats(self):
        """ 게임을 진행하는 동안 바뀔 수 있는 기록을 초기화합니다. """
        self.ships_left = self.settings.ship_limit

이제 UniverseWar 클래스의 _ship_hit() 메서드를 수정하여 플레이어 우주선을 모두 잃었을 때 game_active를 False로 바꿉니다. 

In [ ]:

# universe_war.py

import sys
from time import sleep 
import pygame
# 세팅 모듈 임포트
from settings import Settings
from ship import Ship
from bullet import Bullet
from enemy import Enemy
from game_stats import GameStats

# 게임의 메인 처리부를 담당하는 클래스

class universeWar:
    """게임의 전체의 자원과 공작을 관리하는 클래스"""

    def __init__(self):
        pygame.init()    # 파이게임 초기화

        # Settings 인스턴스 생성
        self.settings = Settings()   # Settings 타입의 객체를 생성해서 멤버 변수에 연결

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)  #해상도
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height

        
        pygame.display.set_caption("Universe War")   # 제목 표시줄 설정
        
        self.stats = GameStates(self)
        self.ship = Ship(self)     # 우주선 객체 생성후 변수에 연결
        self.bullets = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()    # 적 우주선도 그룹으로 묶어서 관리합니다.

        self._create_fleet()     # 적 우주선을 생성합니다.

        # se;f.bg_color = (230, 230, 230)   # 배경색 지정


    def _create_fleet(self):
        """ 적 함대를 만듭니다."""
        enemy = Enemy(self)
        enemy_width = enemy.rect.width           # 외계인 그림의 너비를 가져옵니다. 
        available_space_x = self.settings.screen_width - (2 * enemy_width)   # 가용공간 계산
        number_enemy_x = available_space_x // (2 * enemy_width)      # 한줄에 들어가는 외계인 대수

        # 화면 높이에 앎자은 적 우주선 줄 수를 계산합니다.
        ship_height = self.ship.rect.height    # 플레이어 우주선 높이
        available_space_y = self.(settings.screen_height - (3 * enemy_height) - ship_height) 
        number_rows = avilable_space_y // (2 * enemy_height)

        for row_number in range(number_rows):
            for enemy_number in range(number_enemy_x):
                self._create_enemy(enemy_number, row_number)
        

    def _create_enemy(self, enemy_number, row_number):
        enemy = Enemy(self)
        enemy_width = enemy.rect.width 
        # 생성한 외계인 객체를 출력할 화면상의 x 좌표를 계산합니다.
        enemy.x =  enemy_width + 2 * enemy_width * enemy_number
        enemy.rect.x = enemy.x

        # 적 우주선의 높이만큼 더해서 화면 위쪽에 빈 공간을 만듭니다.
        # 각 줄은 바로 앞 줄보다 적 우주선 높이의 두배 만큼 아래에서 시작하므로 적 우주선 높이에 2를 곱하고 다시 행 번호를 곱합니다.
        # 첫 번째 줄 번호는 0이므로 이 줄의 위치는 변하지 않지만 그 뒤로 이어지는 줄은 모두 화면 아래로 내려가면서 배치합니다.
    

        enemy.rect.y = enemy.rect.height + 2 * enemy.rect.height * row_number
        self.enemies.add(enemy)




    

    def run_game(self):
        while True:
            self._check_events()
       
            # 가장 최근에 그려진 화면을 표시합니다.
            # game_active 플래그가 True 일 때만 우주선을 갱신하고, 탄환을 갱신하고, 적 우주선을 갱신합니다. 
            if self.stats.game_active:
                self.ship.update()
                self._update_bullets()
                self._update_enemies()
                
            self._update_screen()
            
            

            # 사라진 탄환을 제거합니다.
            for bullet in self.bullets.copy():
                if bullet.rect.bottom <= 0:
                    self.bullets.remove(bullet)

            self._check_bullet_enemy_collisions()

           
    def _check_bullet_enemy_collisions(self):
         
        collisions = pygame.sprite.groupcollide(self.bullets, self.enemies, True, True)

                        
        
            if not self.enemies:
                self.bullets.empty()
                self._create_fleet()

    
    # 적 우주선이 바닥에 닿았는지 확인하고 처리합니다.
    def _check_enemies_bottom(self):
        screen_rect = self.screen.get_rect()
        for enemy in self.enemies.sprites():
            if enemy.rect.bottom >= screen_rect.bottom:
                self.ship_hit()
                break

       

    def _update_enemies(self):
        self._check_fleet_edges()
        self.enemies.update()

        # 적 우주선이 플레이어 우주선과 충돌했는지 확인합니다.
        if pygame.sprite.spritecollideany(self.ship, self.enemies):
            self._ship_hit()

        self._check_enemies_bottom()

    def _ship_hit(self):
        """플레이어 우주선이 적 우주선과 충돌했을 떄 반응합니다."""

        if self.stats.ship_left > 0 :
            # 플레이어 우주선 갯수를 줄입니다. 
            self.stats.ships_left -= 1

            # 남아 있는 적 우주선과 탄환을 제거합니다.
            self.enemies.empty()
            self.bullets.empty()

            # 새 함대를 만들고 플레이어 우주선을 배치합니다.
            self._create_fleet()
            self.ship.center_ship()

            # 일시 정지
            sleep(0.5)
        else:
            self.stats.game_active = False
            
    def _check_events(self):
         # 키보드와 마우스 이벤트를 주시합니다.
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:   # 키가 눌렸고
                self._check_keydown_events(event)
                
            elif event.type == pygame.KEYUP:  # 키를 눌렀다가 띄었고 
               self._check_keyup_events(event)

    

    def _check_keydown_events(self, event):
        if event.key == pygame.K_RIGHT:  # 눌린 키가 오른쪽 방향키라면
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:   # 눌린 키가 왼쪽 방향키라면
            elf.ship.moving_left = True
        elif event.key == pygame.k_q:     # 프로그램 실행중 q 키가 입력으로 들어오면 종료
            pygame.quit()
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
            

    def _check_keyup_events(self, event):
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _check_fleet_edges(self):
        """ 적 우주선이 화면 경계에 닿았다면 그에 맞게 반응합니다."""
        for enemy in self.enemies.sprites():
            if enemy.check_edges():
                self._change_fleet_direction()
                break

    def _change_fleet_direction(self):
        for enemy in self.enemies.sprites():
            enemy.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction * = -1

    

    def _fire_bullet(self):
        """새 탄환을 생성하고 bullets 그룹에 추가"""
        if len(self.bullets) < self.settings.bullets.allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

        

    def _update_screen(self):
        self.screen.fill(self.setting.bg_color)

        # 우주선을 그립니다.
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        self.enemies.draw(self.screen)    # draw()를 호출하면 파이게임은 그룹의 각 요소를 화면에 그립니다. 

        pygame.display.flip()
        

# 게임 객체를 생성합니다.
game = UniverseWar()
game.run_game()